In [56]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from pprint import pprint
import io
import json
import re
from collections import OrderedDict
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

In [2]:
# for entity recognition of compounds without cas numbers
def link_compound2chebi(compound):
    """
    used NCBO Annotator from BioPortal to return ChEBI IDS
    for substrates and products of reactions from Expasy enzyme
    """
    url = 'http://data.bioontology.org/annotator'
    params = dict(apikey=api_key, text=compound, ontologies='CHEBI', longest_only='true',
                  include='properties', exlude_numbers='false', exclude_synonyms='false', mappins='all')
    tm_results = requests.get(url=url, params=params)
    return tm_results.json()

In [13]:
def execute_query(query):
    endpoint = SPARQLWrapper('https://query.wikidata.org/sparql')
    endpoint.setQuery(query)
    endpoint.setReturnFormat(JSON)
    return endpoint.query().convert()

# for mapping the cas number to chebi id using wikidata's sparql endpoint
def map_cas_to_chebi_wd(cas_number):
    query = '''
    select ?compoundLabel ?compound ?chebi where {
      ?compound wdt:P231 '%s';
                wdt:P683 ?chebi.
      SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en" .
      }
    }
    ''' % (cas_number)
    results = execute_query(query)
    results = results['results']['bindings']
    return_val = ''
    if len(results) == 1:
        return_val = results[0]
    else:
        return_val = None
    return return_val

In [4]:
# uses pdfminer2 to extract text from a pdf
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [5]:
# extract text to string from a 
document = convert_pdf_to_txt('../data/toc_headings.pdf')

In [6]:
document

"Complete Table of Contents - Volumes One and Two\n\nVolume One\n\nGeneral Information\n\n1 Introduction and Public Health Uses\n4 What’s New and Different?\n6 Data Sources and Data Analysis\n12 Interpretation of Report and Updated Tables Data: Important Factors\n14 Calculation of Urinary Inorganic-related Arsenic Species\n15 Calculation of PFOS and PFOA as the Sum of the Isomers\n17 Chemical and Toxicological Information\n\nAdducts of Hemoglobin\n\n19 Acrylamide\nCAS No. 79-06-1\n20 Glycidamide\nCAS No. 486-56-6\n\nTobacco Smoke\n21 Cotinine\n\nCAS No. 486-56-6\n\n24 Hydroxycotinine\n25 NNAL\n\nCAS No. 76014-81-8\n\nDisinfection By-Products\n\n29 Bromodichloromethane\n\nCAS No. 75-27-4\n\n31 Dibromochloromethane (Chlorodibromomethane)\n\nCAS No. 124-48-1\n\n33 Tribromomethane (Bromoform)\n\nCAS No. 75-25-2\n\n35 Trichloromethane (Chloroform)\n\nCAS No. 67-66-3\n\nPersonal Care and Consumer Product Chemicals and Metabolites\n\n37 Benzophenone-3\n\nCAS No. 131-57-7\n41 Bisphenol A\nCAS 

In [7]:
# start the parsing to filter out lines we don't care about
lines = document.split('\n')
lines = [x for x in lines if x]
lines = [x for x in lines if '_____' not in x]
lines = [x for x in lines if 'Updated Tables' not in x]
lines = [x.lstrip('\x0c') for x in lines if '(cid:' not in x]
lines = lines[9:]

In [8]:
lines

['Adducts of Hemoglobin',
 '19 Acrylamide',
 'CAS No. 79-06-1',
 '20 Glycidamide',
 'CAS No. 486-56-6',
 'Tobacco Smoke',
 '21 Cotinine',
 'CAS No. 486-56-6',
 '24 Hydroxycotinine',
 '25 NNAL',
 'CAS No. 76014-81-8',
 'Disinfection By-Products',
 '29 Bromodichloromethane',
 'CAS No. 75-27-4',
 '31 Dibromochloromethane (Chlorodibromomethane)',
 'CAS No. 124-48-1',
 '33 Tribromomethane (Bromoform)',
 'CAS No. 75-25-2',
 '35 Trichloromethane (Chloroform)',
 'CAS No. 67-66-3',
 'Personal Care and Consumer Product Chemicals and Metabolites',
 '37 Benzophenone-3',
 'CAS No. 131-57-7',
 '41 Bisphenol A',
 'CAS No. 80-05-7',
 '45 Bisphenol F',
 '47 Bisphenol S',
 '49 4-tert-Octylphenol',
 'CAS No. 140-66-9',
 '51 Triclocarban',
 'CAS No. 101-20-2',
 '53 Triclosan',
 'CAS No. 3380-34-5',
 '57 Butyl paraben',
 'CAS No. 94-26-8',
 '61 Ethyl paraben',
 'CAS No. 120-47-8',
 '65 Methyl paraben',
 'CAS No. 99-76-3',
 '69 n-Propyl paraben',
 'CAS No. 94-13-3',
 '73 2,4-Dichlorophenol',
 'CAS No. 120-8

In [24]:
heading_index = OrderedDict()
for index, line in enumerate(lines):
    try:
        if not line[0].isdigit() and not line.startswith('CAS'):
            heading_index[line] = {'index':index,
                                   'chemicals': [],
                                   'no_chebi_cas_numbers':[],
                                   'next_index': '',
                                  }
    except Exception as e:
        print(e)
for key, value in heading_index.items():
    next_key = list(heading_index.keys()).index(key) + 1
    try:
        this_index = value
        next_index = heading_index[list(heading_index.keys())[next_key]]['index']
        heading_index[key]['next_index'] = next_index
    except Exception as e:
        print(e)

string index out of range
list index out of range


In [49]:
heading_index

OrderedDict([('Adducts of Hemoglobin',
              {'index': 0,
               'chemicals': [{'label': 'acrylamide',
                 'chebi': '28619',
                 'cas': '79-06-1'},
                {'label': 'cotinine', 'chebi': '68641', 'cas': '486-56-6'},
                {'label': 'acrylamide', 'chebi': '28619', 'cas': '79-06-1'},
                {'label': 'cotinine', 'chebi': '68641', 'cas': '486-56-6'}],
               'no_chebi_cas_numbers': [],
               'next_index': 5}),
             ('Tobacco Smoke',
              {'index': 5,
               'chemicals': [{'label': 'cotinine',
                 'chebi': '68641',
                 'cas': '486-56-6'},
                {'label': 'cotinine', 'chebi': '68641', 'cas': '486-56-6'},
                {'label': '4-(N-methyl-N-nitrosamino)-1-(3-pyridyl)butan-1-ol',
                 'chebi': '82569',
                 'cas': '76014-81-8'}],
               'no_chebi_cas_numbers': [],
               'next_index': 11}),
             

In [53]:
# start the actual work
for key, value in heading_index.items():
    for index, line in enumerate(lines):
        try:
            if index > int(value['index']) and index < int(value['next_index']):
                if 'CAS' in line:
                    cas = line.lstrip('CAS No. ')
                    chebi_results = map_cas_to_chebi_wd(cas)
                    if chebi_results:
                        value['chemicals'].append({
                                'label': chebi_results['compoundLabel']['value'],
                                'chebi': chebi_results['chebi']['value'],
                                'cas': cas,
                            })  
                    else:
                        value['no_chebi_cas_numbers'].append(line)
        except Exception as e:
            print(line, e)      

679 N-Acetyl-S-(2-carboxyethyl)-L-cysteine invalid literal for int() with base 10: ''
CAS No. 51868-61-2 invalid literal for int() with base 10: ''
681 N-Acetyl-S-(3-hydroxypropyl)-L-cysteine invalid literal for int() with base 10: ''
CAS No. 23127-40-4 invalid literal for int() with base 10: ''
683 N-Acetyl-S-(2-carbamoylethyl)-L-cysteine invalid literal for int() with base 10: ''
CAS No. 81690-92-8 invalid literal for int() with base 10: ''
685 N-Acetyl-S-(2-carbamoyl-2-hydroxyethyl)-L-cysteine invalid literal for int() with base 10: ''
687 N-Acetyl-S-(2-cyanoethyl)-L-cysteine invalid literal for int() with base 10: ''
CAS No. 74514-75-3 invalid literal for int() with base 10: ''
689 N-Acetyl-S-(2-hydroxyethyl)-L-cysteine invalid literal for int() with base 10: ''
CAS No. 15060-26-1 invalid literal for int() with base 10: ''
691 N-Acetyl-S-(phenyl)-L-cysteine invalid literal for int() with base 10: ''
CAS No. 4775-80-8 invalid literal for int() with base 10: ''
693 N-Acetyl-S-(n-prop

In [55]:
json.dump()

OrderedDict([('Adducts of Hemoglobin',
              {'chemicals': [{'cas': '79-06-1',
                              'chebi': '28619',
                              'label': 'acrylamide'},
                             {'cas': '486-56-6',
                              'chebi': '68641',
                              'label': 'cotinine'},
                             {'cas': '79-06-1',
                              'chebi': '28619',
                              'label': 'acrylamide'},
                             {'cas': '486-56-6',
                              'chebi': '68641',
                              'label': 'cotinine'},
                             {'cas': '79-06-1',
                              'chebi': '28619',
                              'label': 'acrylamide'},
                             {'cas': '486-56-6',
                              'chebi': '68641',
                              'label': 'cotinine'},
                             {'cas': '79-06-1',
                

                             {'cas': '319-85-7',
                              'chebi': '28428',
                              'label': 'β-hexachlorocyclohexane'},
                             {'cas': '2385-85-5',
                              'chebi': '34852',
                              'label': 'mirex'},
                             {'cas': '5436-43-1',
                              'chebi': '81584',
                              'label': "2,2',4,4'-tetrabromodiphenyl ether"},
                             {'cas': '68631-49-2',
                              'chebi': '81534',
                              'label': "2,2',4,4',5,5'-hexabromodiphenyl "
                                       'ether'},
                             {'cas': '207122-15-4',
                              'chebi': '81535',
                              'label': "2,2',4,4',5,6'-hexabromodiphenyl "
                                       'ether'},
                             {'cas': '207122-16-5',
              

               'no_chebi_cas_numbers': ['CAS No. 309-00-2',
                                        'CAS No. 72-20-8',
                                        'CAS No. 1024-57-3']}),
             ('Polybrominated Diphenyl Ethers and PBB 153 (Pooled Samples '
              'after 2004)',
              {'chemicals': [{'cas': '5436-43-1',
                              'chebi': '81584',
                              'label': "2,2',4,4'-tetrabromodiphenyl ether"},
                             {'cas': '68631-49-2',
                              'chebi': '81534',
                              'label': "2,2',4,4',5,5'-hexabromodiphenyl "
                                       'ether'},
                             {'cas': '207122-15-4',
                              'chebi': '81535',
                              'label': "2,2',4,4',5,6'-hexabromodiphenyl "
                                       'ether'},
                             {'cas': '207122-16-5',
                              'chebi'